In [1]:
import os
from datetime import datetime

import numpy as np

## Define input and output filepaths

In [2]:
MATTERPORT_DIR = "/media/rsl_admin/T7/matterport/data/v1/scans"

LATTICE_GRAPHS_DIR = "/media/rsl_admin/T7/matterport/lattice_graphs"

In [6]:
OPENSCENE_REGION_SEGS_DIR = \
    "/home/rsl_admin/openscene/comparison_outputs/segmentations/region-openseg-matterport-test"

# OPENSCENE_REGION_SEGS_DIR = \
#     "/home/rsl_admin/openscene/comparison_outputs/segmentations/region-openseg-matterport-val"

In [7]:
OUTPUT_DIR = "/home/rsl_admin/openscene/comparison_outputs/dbscan_param_sweeps_outputs"


## Figure out which scans to process

In [8]:
scans = set()
for fname in os.listdir(OPENSCENE_REGION_SEGS_DIR):
    scans.add(fname.split("_")[0])
    
scans = tuple(sorted(scans))

print(scans)

('2t7WUuJeko7', '5ZKStnWn8Zo', 'ARNzJeq3xxb', 'RPmz2sHmrrY', 'UwV83HsGsw3', 'Vt2qJdWjCF2', 'WYY7iVyf5p8', 'YFuZgdQ5vWj', 'YVUC4YcDtcY', 'fzynW3qQPVF', 'gYvKGZ5eRqb', 'gxdoqLR6rwA', 'jtcxE69GiFV', 'pa4otMbVnkk', 'q9vSo1VnCiC', 'rqfALeAoiTq', 'wc2JMjhGNzB', 'yqstnuAEVhm')


## Evaluation parameters

In [9]:
BLACKLISTED_REGION_LABELS = (
    "other room",

    # no appropriate tag
    "dining booth",
    "entryway/foyer/lobby",
    "outdoor",
)

In [10]:
params = {
        "label_params": {
            "blacklisted_labels": BLACKLISTED_REGION_LABELS,
        },
        # Make sure sides of proposal boxes are at least this long
        "min_proposal_box_length": 0.1,
    
    
        # From email with the author of OVIR
        "cluster_min_points": 50,

        "dbscan_params": {
            "eps": None,
            "min_points": None,
            "print_progress": False,
        },
    }

## Generate sweeped DBSCAN values

In [11]:
EPS_RANGE = [0.5, 1.5]
EPS_INCREMENT = 0.5

eps_values = np.arange(
    (EPS_RANGE[1] - EPS_RANGE[0])/ EPS_INCREMENT + 1
) * EPS_INCREMENT + EPS_RANGE[0]

print(eps_values)

[0.5 1.  1.5]


In [12]:
MIN_POINTS_RANGE = [1, 1]
MIN_POINTS_INCREMENT = 1

min_points_values = np.arange(
    (MIN_POINTS_RANGE[1] - MIN_POINTS_RANGE[0])/ MIN_POINTS_INCREMENT + 1
) * MIN_POINTS_INCREMENT + MIN_POINTS_RANGE[0]

print(min_points_values)

[1.]


In [13]:
X, Y = np.meshgrid(eps_values, min_points_values)
combinations = np.vstack([X.ravel(), Y.ravel()]).T

combinations = [tuple(row) for row in combinations]

print(combinations)

[(0.5, 1.0), (1.0, 1.0), (1.5, 1.0)]


## Run evaluation on the scans in the segmentations directory

In [14]:
from evaluate_matterport_scan_region import evaluate_matterport_scan_region_localizations

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [15]:
for combo in combinations:
    eps, min_points = combo
    params["dbscan_params"]["eps"] = float(eps)
    params["dbscan_params"]["min_points"] = int(min_points)

    
    print(params)
    
    
    save_dir = os.path.join(
        OUTPUT_DIR, f"region_eps-{eps}_min_points-{min_points}")
    
    os.makedirs(save_dir, exist_ok=False)
    
    
    for scan_name in scans:

        scan_dir = os.path.join(MATTERPORT_DIR, f"{scan_name}/{scan_name}")
        lattice_graph_path = os.path.join(LATTICE_GRAPHS_DIR, f"{scan_name}_lattice_graph.pkl")

        evaluate_matterport_scan_region_localizations(
            params=params,
            scan_dir=scan_dir,
            openscene_region_segs_dir=OPENSCENE_REGION_SEGS_DIR,
            lattice_graph_path=lattice_graph_path,
            output_dir=save_dir
        )

running evaluation on matterport scan 2t7WUuJeko7
loaded house file from /media/rsl_admin/T7/matterport/data/v1/scans/2t7WUuJeko7/2t7WUuJeko7/house_segmentations/2t7WUuJeko7.house
loaded ply file from /media/rsl_admin/T7/matterport/data/v1/scans/2t7WUuJeko7/2t7WUuJeko7/house_segmentations/2t7WUuJeko7.ply
found the following region segmentation files for scan 2t7WUuJeko7:
2t7WUuJeko7_region4.pkl
	2t7WUuJeko7_region3.pkl
	2t7WUuJeko7_region5.pkl
	2t7WUuJeko7_region1.pkl
	2t7WUuJeko7_region0.pkl
	2t7WUuJeko7_region2.pkl


{'label_params': {'blacklisted_labels': ('other room', 'dining booth', 'entryway/foyer/lobby', 'outdoor')}, 'min_proposal_box_length': 0.1, 'cluster_min_points': 50, 'dbscan_params': {'eps': 0.5, 'min_points': 1, 'print_progress': False}}


loaded lattice graph from /media/rsl_admin/T7/matterport/lattice_graphs/2t7WUuJeko7_lattice_graph.pkl
aggregated region segmentations
started localization pipeline
100%|█████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.47it/s]
finished running localization pipeline
/home/rsl_admin/openscene/openscene_env/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:354: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add an explicit dependency on ECOS or switch to our new
    default solver, Clarabel, by either not specifying a solver argument
    or specifying ``solver=cp.CLARABEL``.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)
started computing precision metrics
  0%|                                                                         | 0/5 [00:00<?, ?it/s]proposal with no assigned lattice no

proposal with no assigned lattice nodes in closet, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in closet, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in closet, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in closet, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in closet, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in closet, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in hallway, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in hallway, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in hallway, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in hallway, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in hal

proposal with no assigned lattice nodes in hallway, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in dining room, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in dining room, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in dining room, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in kitchen, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in kitchen, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in kitchen, setting mean shortest path length to np.nan
100%|█████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 24.31it/s]
finished computing precision metrics
finished computing precision metrics
finished computing precision metrics
started computing recall metrics
started computing recall metrics
started computing recall metrics
finis

...

found the following region segmentation files for scan yqstnuAEVhm:
yqstnuAEVhm_region3.pkl
	yqstnuAEVhm_region25.pkl
	yqstnuAEVhm_region2.pkl
	yqstnuAEVhm_region20.pkl
	yqstnuAEVhm_region14.pkl
	yqstnuAEVhm_region27.pkl
	yqstnuAEVhm_region15.pkl
	yqstnuAEVhm_region18.pkl
	yqstnuAEVhm_region23.pkl
	yqstnuAEVhm_region22.pkl
	yqstnuAEVhm_region11.pkl
	yqstnuAEVhm_region5.pkl
	yqstnuAEVhm_region6.pkl
	yqstnuAEVhm_region0.pkl
	yqstnuAEVhm_region7.pkl
	yqstnuAEVhm_region13.pkl
	yqstnuAEVhm_region10.pkl
	yqstnuAEVhm_region17.pkl
	yqstnuAEVhm_region12.pkl
	yqstnuAEVhm_region24.pkl
	yqstnuAEVhm_region1.pkl
	yqstnuAEVhm_region16.pkl
	yqstnuAEVhm_region8.pkl
	yqstnuAEVhm_region9.pkl
	yqstnuAEVhm_region4.pkl
	yqstnuAEVhm_region28.pkl
	yqstnuAEVhm_region26.pkl
	yqstnuAEVhm_region19.pkl
	yqstnuAEVhm_region21.pkl
found the following region segmentation files for scan yqstnuAEVhm:
yqstnuAEVhm_region3.pkl
	yqstnuAEVhm_region25.pkl
	yqstnuAEVhm_region2.pkl
	yqstnuAEVhm_region20.pkl
	yqstnuAEVhm_region1

loaded lattice graph from /media/rsl_admin/T7/matterport/lattice_graphs/yqstnuAEVhm_lattice_graph.pkl
loaded lattice graph from /media/rsl_admin/T7/matterport/lattice_graphs/yqstnuAEVhm_lattice_graph.pkl
loaded lattice graph from /media/rsl_admin/T7/matterport/lattice_graphs/yqstnuAEVhm_lattice_graph.pkl
loaded lattice graph from /media/rsl_admin/T7/matterport/lattice_graphs/yqstnuAEVhm_lattice_graph.pkl
loaded lattice graph from /media/rsl_admin/T7/matterport/lattice_graphs/yqstnuAEVhm_lattice_graph.pkl
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated region segmentations
aggregated

proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office,

proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office, setting mean shortest path length to np.nan
proposal with no assigned lattice nodes in office,

saved evaluation outputs to /home/rsl_admin/openscene/comparison_outputs/dbscan_param_sweeps_outputs/region_eps-1.5_min_points-1.0/yqstnuAEVhm.pkl
saved evaluation outputs to /home/rsl_admin/openscene/comparison_outputs/dbscan_param_sweeps_outputs/region_eps-1.5_min_points-1.0/yqstnuAEVhm.pkl
saved evaluation outputs to /home/rsl_admin/openscene/comparison_outputs/dbscan_param_sweeps_outputs/region_eps-1.5_min_points-1.0/yqstnuAEVhm.pkl
saved evaluation outputs to /home/rsl_admin/openscene/comparison_outputs/dbscan_param_sweeps_outputs/region_eps-1.5_min_points-1.0/yqstnuAEVhm.pkl
saved evaluation outputs to /home/rsl_admin/openscene/comparison_outputs/dbscan_param_sweeps_outputs/region_eps-1.5_min_points-1.0/yqstnuAEVhm.pkl
saved evaluation outputs to /home/rsl_admin/openscene/comparison_outputs/dbscan_param_sweeps_outputs/region_eps-1.5_min_points-1.0/yqstnuAEVhm.pkl
saved evaluation outputs to /home/rsl_admin/openscene/comparison_outputs/dbscan_param_sweeps_outputs/region_eps-1.5_mi